In [ ]:
import numpy as np
import pandas as pd
import sys, os
from collections import defaultdict
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

sys.path.append('../Shared')
import processing, Model, Visualization


def call_data(base_path, sub_lst, sub_idx):
    data = pd.read_csv(base_path+sub_lst[sub_idx])
    return data

def data_setup(lst_normal, path_normal, window_size, step_size):
    subjects_data = defaultdict(list)
    subjects_label = defaultdict(list)

    for s_idx, filename in enumerate(lst_normal):
        name = filename.lower()
        subject_id = name.split("n")[0]
        data = call_data(path_normal, lst_normal, s_idx).to_numpy()  # (num_samples, num_channels)

        windows = processing.sliding_window(data, window_size, step_size)  # (num_windows, win_len, ch)

        if "standing" in name: label = 0
        elif "gait" in name: label = 1
        elif "sitting" in name: label = 2
        else: label = -1

        labels = np.full(len(windows), label)

        subjects_data[subject_id].append(windows)
        subjects_label[subject_id].append(labels)

    return subjects_data, subjects_label



def get_X_y(X, y):
    all_X, all_y = [], []

    #for subject_id in subjects_data.keys():
    data_list = X
    label_list = y

    for data, labels in zip(data_list, label_list):
        for w, label in zip(data, labels):  # w: (win_len, ch)
            feat = processing.extract_features(w)  # (num_channels*5,)
            all_X.append(feat)
            all_y.append(label)

    all_X, all_y = np.array(all_X), np.array(all_y)   # (N, num_channels*5)

    return all_X, all_y

In [ ]:
path_abnormal = "../../../Data/Gait1/Abnormal/"
path_normal = "../../../Data/Gait1/normal/"

lst_abnormal = os.listdir(path_abnormal)
lst_normal = os.listdir(path_normal)

window_size, step_size = 200, 10
num_channels, num_features = 5, 5  # we extracted 5 features per channel

subjects_data, subjects_label = data_setup(lst_normal, path_normal, window_size, step_size)

for subject_id in subjects_data.keys():
    X, y = get_X_y(X=subjects_data[subject_id], y=subjects_label[subject_id])
    X = X.reshape(-1, num_channels, num_features, 1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = Model.build_model(input_shape=X_train.shape[1:], num_classes=len(np.unique(y)))
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=128, verbose=0)
    #Visualization.learning_plot(history)
    print(f"Sub ID {subject_id} : val acc => {np.max(history.history['val_accuracy'])*100:.2f}%")